# Halloween 🎃

Here is my example. This notebook reads all the images in the `Images` folder and displays a random one when something gets too close. Check the comments for details.

Things we need for the sensor and LCD display.

In [ ]:
import time
from IPython.display import clear_output
from PIL import Image
from ltr559 import LTR559
import ST7735

ltr559 = LTR559()

disp = ST7735.ST7735(port=0,cs=1,dc=9,backlight=12,rotation=270,spi_speed_hz=10000000)
disp.begin()

This routine will show an image gallery given a list of file names.

In [ ]:
from IPython.display import HTML

def gallery(images, row_height='auto'):
    figures = []
    for image in images:
        src = image
        caption = f'<figcaption style="font-size: 0.6em">{image}</figcaption>'
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{src}" style="height: {row_height}">
              {caption}
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

#### Reading all images and store them in list

Now we use the command glob to get a list of all `images` in the Images folder. Then we read and resize them and add them to our list of images for later.

In [ ]:
from glob import glob

# First get a list of all files
files = glob('Images/*')
print ("Found {0:4d} image files.".format(len(files)))

# Now loop through the files, read and store them in list images
images=[]
for file in files:
    image = Image.open(file)
    image.thumbnail([disp.width, disp.height])    
    images.append(image)
    
gallery(files, row_height="80px")

#### Main loop

This is the loop checking proximity. Closer than 50 it shows a random image. Note that we select the random image when proximity is smaller that the threshhold, otherwise it woudl keep changing while an object is close 😮 

In [ ]:
import random

# Just in case something is already close, set the image to a random one.
# Note the random number gord from zero to one less that the length of the list
image = images[random.randint(0,len(images)-1)] 

try:
    while True:
        ltr559.update_sensor()
        prox = ltr559.get_proximity()

        clear_output(wait=True)
        print(prox)
        
        if(prox > 50):
            img = Image.new('RGBA', (disp.width, disp.height))
            img.paste(image,(int((disp.width-image.width)/2),0))  # this code puts it in the middle of the screen
            disp.display(img)
        else:
            image = images[random.randint(0,len(images)-1)]       # This sets images to a new random image
            img = Image.new('RGBA', (disp.width, disp.height))
            disp.display(img)
            
        time.sleep(0.5)
        
except KeyboardInterrupt:
    pass